Imports

In [1]:
import numpy as np
import random

Other Functions

In [2]:
def getRandThousandths() -> float :
    return random.randrange( 1, 999, 1 ) / 1000

def accuracy( array1:list, array2:list ) :
    assert len( array1 ) == len( array2 ), f'Array Lengths Must Be Equal For Accuracy\n\t{ array1 } != { array2 }'
    
    totalIncorrect = 0.0
    
    for i in range( 0, len( array1 ) ) :
        if array1[ i ] != array2[ i ] : totalIncorrect += 1
        
    return totalIncorrect / len( array1 )
    
def sqrtDiff( array1:list, array2:list ) :
    assert len( array1 ) == len( array2 ), f'Array Lengths Must Be Equal For Square Root Difference\n\t{ array1 } != { array2 }'
    
    error = 0.0
    
    for i in range( 0, len( array1 ) ) :
        error += np.sqrt( ( array1[ i ] * array1[ i ] ) + ( array2[ i ] * array2[ i ] ) )
            
    return error / len( array1 )

Activation Functions Class

In [3]:
def tanh( x ) :
    return np.tanh( x )

def dtanh( x ) :
    return 1 - ( tanh( x ) * tanh( x ) )

Layer Class

In [4]:
class Layer :
    
    def __init__( self, inputShape:list,  numberOfNodes:int, activationFunction:callable, weightsRandomizer:callable ) -> None:
        
        assert numberOfNodes > 0, "A Layer Must Contain More Than 0 Nodes"
        
        self.inputShape = inputShape
        self.numberOfNodes = numberOfNodes
        self.outputShape = numberOfNodes
        self.activationFunction = activationFunction
        
        self.weights = []
        self.initWeights( numberOfNodes, weightsRandomizer )
    
    def initWeights( self, numberOfNodes, weightsRandomizer ) :
        
        w = np.random.randn( self.inputShape + 1, self.outputShape + 1 )
        
        self.weights = w / np.sqrt( self.inputShape)        

        # for i in range( 0, self.inputShape ):

        #     weightsPerInput = []

        #     for node in range( 0, numberOfNodes ) :
        #         weightsPerInput.append( weightsRandomizer() )

        #     weightsPerInput.append()

        #     self.weights.append(weightsPerInput)


        #Bias
        # self.weights.append( random.randrange( 1, 999, 1 ) / 1000 )

Neural Net Class

In [5]:
class NeuralNet :
    
    def __init__( self, loss:callable = sqrtDiff, weightsRandomizer:callable = getRandThousandths ) -> None :
        self.layers = []
        self.weightsRandomizer=weightsRandomizer
        self.lossFunc = loss
    
    def addLayer( self, inputShape=None, numberOfNodes=0, activationFunction:callable=tanh ) -> None :
        
        if len( self.layers ) == 0 :
                        
            assert type( inputShape ) == int
            assert inputShape > 0
            
            self.layers.append( Layer( inputShape, numberOfNodes, activationFunction, self.weightsRandomizer ) )
            
        else : 
            self.layers.append( Layer( self.layers[ -1 ].outputShape, numberOfNodes, activationFunction, self.weightsRandomizer ) )
  
    def sigmoid_transfer( self, output ) :
        return output * ( 1 - output )        
        
            
    def trainModel( self, input, correctData, metric:callable=None ) :
        
        assert type( input ) == list, "Input Data For Testing Must Be a List"
        assert type( correctData ) == list, "Correct Answers For Testing Must Be a List"
        
        #Multiple Training Examples
        if all( isinstance( item, list ) for item in input ) :
                        
            predictions = []
            
            for example in input :
                assert len( example ) == self.layers[0].inputShape, "Training Examples Must Be of size : " + str( self.layers[0].inputShape )
                predictions.append( self.forwardPropigate( example ) )
        
            metricResult = self.checkMetric( predictions, correctData )
            return predictions, metricResult
        
        #Single training Example    
        else : 
            
            assert len( input ) == self.layers[0].inputShape, "Training Example Must Be of size : " + str( self.layers[0].inputShape )
            prediction = self.forwardPropigate( input )
            
            metricResult = self.checkMetric( prediction, correctData )
            print(metricResult)
            return prediction, metricResult
    
    def testModel( self, input ) :
        
        assert type( input ) == list, "Input Data For Testing Must Be a List"
        
        #Multiple Training Examples
        if all( isinstance( item, list ) for item in input ) :
                        
            predictions = []
            
            for example in input :
                assert len( example ) == self.layers[0].inputShape, "Training Examples Must Be of size : " + str( self.layers[0].inputShape )
                predictions.append( self.forwardPropigate( example ) )
        
            return predictions
        
        #Single training Example    
        else : 
            
            assert len( input ) == self.layers[0].inputShape, "Training Example Must Be of size : " + str( self.layers[0].inputShape )
            prediction = self.forwardPropigate( input )
                    
            return prediction
    
    def forwardPropigate( self, testData ) :
        
        passDownArray = testData            
                
        for layer in self.layers :
            
            passDownArray = self.activationFunction( np.matmul( passDownArray, layer.weights ), layer )
                                
        return passDownArray
    
    def activationFunction( self, array, layer ) :
        return [ layer.activationFunction( x )  for x in array ]
          
    def checkMetric( self, predictionArray:list, answerArray:list, metric:callable=None ) :
        
        if metric == None : return self.lossFunc( predictionArray, answerArray )
        else : return metric( predictionArray, answerArray )
            
    def printLayer( self, layer ) :
        print( "Input Shape :", layer.inputShape, ': Output Shape :', layer.outputShape, ": Activation Function :", layer.activationFunction )
    
    def printNetwork( self ) :
        
        for layer in self.layers :
            print( "Input Shape :", layer.inputShape,
                  ': Output Shape :', layer.outputShape,
                  ": Activation Function :", layer.activationFunction,
                  "\n\tLayer Weights :", layer.weights
                )

Test Add Layer

In [6]:
N = NeuralNet()

N.addLayer( inputShape=3, numberOfNodes=2 )
N.addLayer( numberOfNodes=3 )
N.addLayer( numberOfNodes=2 )
N.addLayer( numberOfNodes=1 )

N.printNetwork()

print( 'Result =', N.trainModel( [ 1, 1, 1 ], [ 1 ] ) )
# N.testModel( [ [ 1, 1, 1], [ 0.5, 0.5, 0.5] ] )

Input Shape : 3 : Output Shape : 2 : Activation Function : <function tanh at 0x0000012EBACDE040> 
	Layer Weights : [[ 1.15166922 -0.35063491 -0.12706866]
 [ 0.15598088 -0.69955787 -1.18394052]
 [-1.0760472   0.44118259  0.09926373]
 [-0.25203149  0.01804891 -0.67423979]]
Input Shape : 2 : Output Shape : 3 : Activation Function : <function tanh at 0x0000012EBACDE040> 
	Layer Weights : [[ 0.99031251 -0.25595901 -0.21002197 -1.24067015]
 [ 0.23778445 -0.22389616 -0.16517117 -1.20140214]
 [ 0.74102115  0.29728229 -0.76755629  0.55818772]]
Input Shape : 3 : Output Shape : 2 : Activation Function : <function tanh at 0x0000012EBACDE040> 
	Layer Weights : [[-0.26120922 -0.51405569  0.43544047]
 [-0.37397311  0.08967413  0.172106  ]
 [-0.30142258 -0.73219161 -1.10876833]
 [ 0.43027369  0.47187802 -0.03097659]]
Input Shape : 2 : Output Shape : 1 : Activation Function : <function tanh at 0x0000012EBACDE040> 
	Layer Weights : [[ 1.60833111 -0.39090556]
 [ 0.64481402 -0.9144462 ]
 [ 0.37011144  0.2

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 3)